# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [55]:
import pandas as pd
import sys
import warnings
warnings.filterwarnings('ignore')

sys.path.append("../../05_src")

from logger import get_logger
_logs = get_logger(__name__)


columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

#_logs.info("Adult data set : \n %s", adult_dt.head())
print("Adult data set : \n", adult_dt.head())

Adult data set : 
    age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0              40   U

In [52]:
#_logs.info("Adult data set : \n %s", adult_dt.columns)
print("Adult data set Columns : \n %s", adult_dt.columns)

Adult data set : 
 %s Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [56]:
Y = adult_dt['income']
X = adult_dt[adult_dt.columns[:-1]]

X,Y

(       age          workclass  fnlwgt    education  education-num  \
 0       39          State-gov   77516    Bachelors             13   
 1       50   Self-emp-not-inc   83311    Bachelors             13   
 2       38            Private  215646      HS-grad              9   
 3       53            Private  234721         11th              7   
 4       28            Private  338409    Bachelors             13   
 ...    ...                ...     ...          ...            ...   
 32556   27            Private  257302   Assoc-acdm             12   
 32557   40            Private  154374      HS-grad              9   
 32558   58            Private  151910      HS-grad              9   
 32559   22            Private  201490      HS-grad              9   
 32560   52       Self-emp-inc  287927      HS-grad              9   
 
             marital-status          occupation    relationship    race  \
 0            Never-married        Adm-clerical   Not-in-family   White   
 1      

In [57]:
# !pip install scikit-learn
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*Answer*

1) 'random_state' helps us get the same split of our data between runs.
2) Setting random state is useful for getting consistent results when we run our code multiple times, making it easier to understand the changes we make between experiments.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [59]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline


#Assign numerical features columns and categorical features columns:
num_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_features = ['workclass','education','marital-status','occupation', 'relationship', 'race', 'sex','native-country']

#create transformer for numerical features:
num_transformer = Pipeline(
    steps=[("knnimputer", KNNImputer(n_neighbors=7, weights='distance')), ("robust_scaler", RobustScaler())]
)

#create transformer for categorical features:
cat_transformer = Pipeline(
    steps=[
        ("simple_imputer", SimpleImputer(strategy='most_frequent')),
        ("onehot", OneHotEncoder(drop='if_binary', handle_unknown='infrequent_if_exist')),
    ]
)

#create columntransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num_transforms", num_transformer, num_features),
        ("cat_transforms", cat_transformer, cat_features),
    ]
)

preprocessor

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('knnimputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [60]:
from sklearn.ensemble import RandomForestClassifier

pipe = Pipeline(
    [
        ('preprocessing', preprocessor),
        ('classifier', RandomForestClassifier())
    ]
)
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('robust_scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [61]:
from sklearn.model_selection import cross_validate

scoring = ['neg_log_loss', 'roc_auc','accuracy', 'balanced_accuracy']
cv = cross_validate(pipe, X_train, Y_train, cv=5, scoring = scoring, return_train_score = True)
cv

/opt/miniconda3/envs/scale2prod/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/scale2prod/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


{'fit_time': array([3.29566407, 3.28419185, 3.29112387, 3.28267312, 3.2117703 ]),
 'score_time': array([0.11939192, 0.11250305, 0.11312795, 0.11049771, 0.1102047 ]),
 'test_neg_log_loss': array([-0.3657189 , -0.35356306, -0.39525815, -0.36941754, -0.37329536]),
 'train_neg_log_loss': array([-0.08130925, -0.08117062, -0.08159361, -0.08245685, -0.08178912]),
 'test_roc_auc': array([0.90318152, 0.90229674, 0.90210257, 0.90719173, 0.90274271]),
 'train_roc_auc': array([1., 1., 1., 1., 1.]),
 'test_accuracy': array([0.85040579, 0.84821233, 0.85190873, 0.85892935, 0.85607723]),
 'train_accuracy': array([1.        , 0.99994515, 1.        , 1.        , 0.99989031]),
 'test_balanced_accuracy': array([0.77649045, 0.77074054, 0.77303034, 0.78289467, 0.77732   ]),
 'train_balanced_accuracy': array([1.        , 0.9999638 , 1.        , 1.        , 0.99977386])}

In [ ]:
pd.DataFrame(cv).sort_values(by='test_neg_log_loss')

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [ ]:
cv_df = pd.DataFrame(cv)
cv_df.sort_values(by='test_neg_log_loss')

Calculate the mean of each metric. 

In [ ]:
pd.DataFrame(cv_df.mean(), columns=['Mean'])

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

pipe.fit(X_train, Y_train)
Y_predict = pipe.predict(X_test)
Y_pred_proba = pipe.predict_proba(X_test)

performance_metrics = {
        'neg_log_loss': -log_loss(Y_test, Y_pred_proba),
        'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
        'accuracy': accuracy_score(Y_test, Y_predict),
        'balanced_accuracy': balanced_accuracy_score(Y_test, Y_predict)
    }

pd.DataFrame(performance_metrics, index=[0]).T.style.hide(axis='columns').set_caption("<h3>Performance_metrics")

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

**Answer**

we are choosing an arbitrary threshold of $50,000 in income as the most relevant indicator and assigning 0 or 1 value.
By making it a binary target, we simplify the application of certain models while also facilitating a clearer understanding and explanation of their results.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.